In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pytz

In [ ]:
##TODO: META, SHOP

### Pickles raw data dir

In [ ]:
!ls ../data/pickles

### Acceptable frequencies

In [ ]:
minute_frequencies_conventions = {
    5: '5T', 
    15: '15T',
}

### Stock for resampling and target frequency

In [ ]:
stock_tickers = ['QCOM', 'NVDA', 'AMZN', 'MSFT', 'GOOG', 'TSLA', 'AMD', 'INTC', 'NFLX', 'BAC', 'GS', 'JPM', 'WFC', 'MA', 'SQ', 'PYPL']

stock_ticker = stock_tickers[1]
print(stock_ticker)
frequency_int = 1

frequency = minute_frequencies_conventions[frequency_int]

### Paths

In [ ]:
stock = pd.read_pickle(f'../data/pickles/{stock_ticker}.pickle.gzip')
output = f'../data/resampled_data/{frequency}/{stock_ticker}_resampled_{frequency}.csv'

### Stock split options

In [ ]:
match stock_ticker:
    case 'NVDA':
        split_stock = True
        split_date = '2021-07-20'
        split_ratio = 4/1
    case 'AMZN':
        split_stock = True
        split_date = '2022-06-06' # +2
        split_ratio = 20/1
    case 'GOOG':
        split_stock = True
        split_date = '2022-07-17' # +2
        split_ratio = 20/1
    case 'TSLA':
        split_stock = True
        split_date = '2022-08-25'
        split_ratio = 3/1
    case _  :
        split_stock = False
        

### Trading histogram by hours

In [ ]:
hours = pd.DataFrame(stock.index.hour, columns=['hour'])

hourly_counts = hours.groupby('hour').size()

counts_df = pd.DataFrame(hourly_counts).reset_index()
counts_df.columns = ['hour', 'count']
counts_df['cumulative_count'] = counts_df['count'].cumsum()
print(counts_df)

In [ ]:
stock.head(10)

In [ ]:
len(stock)

### Drop records before 2020 (data include some 1970s rows)

In [ ]:
stock = stock.sort_index()
stock = stock.loc['2020':]

#2021-05-16'

### Drop trailing sequences in volume column

In [ ]:
# Shift the hours back to NY time
source_tz = 'UTC'
target_tz = 'America/New_York'  # This is 4 hours behind UTC
stock.index = stock.index.tz_localize(source_tz).tz_convert(target_tz).tz_localize(None)

In [ ]:
# import pytz

source_tz = pytz.timezone('Europe/Prague')
target_tz = pytz.timezone('America/New_York')

stock.index = source_tz.localize(stock.index)
new_york_time = source_tz.astimezone(target_tz).normalize()

In [ ]:
mask = stock['v'] != stock['v'].shift()
stock.loc[~mask, 'v'] = 0

In [ ]:
# Assuming you have a DataFrame 'stock' with a 'volume' column
hours_vol = stock.groupby(stock.index.hour).agg({'v': 'sum'}).reset_index()

hourly_volume_sums = stock.groupby(stock.index.hour)['v'].sum()
# Create a histogram
plt.bar(hours_vol['index'], hours_vol['v'])
plt.xlabel('Hour')
plt.ylabel('Total Volume Sum')
plt.title('Volume Sum by Hour')
plt.show()

### Resample by frequency

In [ ]:
resampled_df = pd.DataFrame()

# Calculate mid-price and spread
stock['mid'] = (stock['P'] + stock['p']) / 2
stock['spread'] = stock['P'] - stock['p']

# First opening price in each 15 minutes
resampled_df['open'] = stock['mid'].resample(frequency).first()  

# Last closing price in each 15 minutes
resampled_df['close'] = stock['mid'].resample(frequency).last()  

# Minimum price in each 15 minutes
resampled_df['low'] = stock['mid'].resample(frequency).min()  

# Maximum price in each 15 minutes
resampled_df['high'] = stock['mid'].resample(frequency).max() 

# Sum of the volumes in the diven time window interval
resampled_df['vol'] = stock['v'].resample(frequency).sum()

# Bid-ask spread in each 15 minutes
resampled_df['spread'] = stock['spread'].resample(frequency).mean()

# Add ticker column
resampled_df['ticker'] = stock_ticker

### Normalize prices based on stock's ratio split from a split date

In [ ]:
plt.style.use('ggplot')

color = 'tab:blue'

plt.figure(100, figsize=(18, 5))

plt.title(f'NVDA Before Stock Split')

sns.lineplot(x = resampled_df.index, y = "close", data = resampled_df, color=color, label=stock_ticker, linewidth=1);

In [ ]:
if split_stock:
    stock_split_date = pd.Timestamp(split_date)
    before_split = resampled_df.index < stock_split_date
    resampled_df.loc[before_split, ['open', 'close', 'low', 'high']] /= split_ratio
    resampled_df.loc[before_split, 'vol'] *= split_ratio

In [ ]:
plt.style.use('ggplot')

color = 'tab:blue'

plt.figure(figsize=(18, 5))

plt.title(f'NVDA After Stock Split')

sns.lineplot(x = resampled_df.index, y = "close", data = resampled_df, color=color, label=stock_ticker, linewidth=1);

### Filtering

In [ ]:
# Filter out weekends (Saturday = 5, Sunday = 6)
resampled_df = resampled_df[resampled_df.index.dayofweek < 5]

# Identify days with no valid prices for the whole day
no_price_days = resampled_df.groupby(resampled_df.index.normalize())['open'].transform(lambda x: x.isna().all())

# add trailing volumes back
resampled_df['vol'].replace(to_replace=0, method='ffill', inplace=True)

# Keep only records for times (8:00 - 24:00)
resampled_df = resampled_df.between_time('08:00:00', '23:59:59')

# Keep only records from active trade times
resampled_df = resampled_df.between_time('09:30:00', '15:59:00')

# Start from 2021-04-17
resampled_df = resampled_df.loc['2021-04-17': '2023-09-01 15:59:59']

# Remove rows for days with no valid prices
resampled_df = resampled_df[~no_price_days]

# Forward fill missing hours
resampled_df = resampled_df.fillna(method='ffill')

# Backward fill missing hours
resampled_df = resampled_df.fillna(method='bfill')

In [ ]:
hours = pd.DataFrame(resampled_df.index.hour, columns=['hour'])

hourly_counts = hours.groupby('hour').size()

counts_df = pd.DataFrame(hourly_counts).reset_index()
counts_df.columns = ['hour', 'count']
counts_df['cumulative_count'] = counts_df['count'].cumsum()
print(counts_df)

In [ ]:
resampled_df['sma_2h'] = resampled_df['open'].rolling(window=int(2*60/frequency_int)).mean()
resampled_df['sma_8h'] = resampled_df['open'].rolling(window=int(8*60/frequency_int)).mean()

resampled_df[['sma_2h', 'sma_8h']] = resampled_df[['sma_2h', 'sma_8h']].fillna(0)

In [ ]:
resampled_df.tail(10)

In [ ]:
plt.figure(100, figsize=(15, 5))
sns.lineplot(x = resampled_df.index, y = "close", data = resampled_df, palette="coolwarm", label=stock_ticker);

In [ ]:
plt.figure(100, figsize=(20, 7))
sns.lineplot(x = resampled_df.index, y = "vol", data = resampled_df, palette="coolwarm", label=f'{stock_ticker} volume');

### Tests

In [ ]:
num_of_days = len(resampled_df.groupby(resampled_df.index.date))
number_of_records = len(resampled_df)

expected_number_of_days = 601
expected_number_of_records = int((num_of_days*6.5*(60/frequency_int))) # last day till 15:59:59, 8 hours missing till 23:59:59

In [ ]:
print(f'number of days: {num_of_days}')
print(f'number of records: {number_of_records}')

print('')

print(f'expected number of days: {expected_number_of_days}')
print(f'expected number of records: {expected_number_of_records}')

In [ ]:
assert num_of_days == expected_number_of_days
assert number_of_records == expected_number_of_records
assert resampled_df.isna().sum().sum() == 0

In [ ]:
resampled_df.to_csv(output, sep=';')